
Swiss-roll projection: alpha filtration persistent homology and persistence rings.

This example generates a noisy two dimensional projection of a classical Swiss-roll
point cloud, computes persistent homology over $\mathbb F_2$ using an alpha
filtration induced by a Delaunay triangulation, and visualizes the resulting barcode
via persistence rings.

# Data model

## Swiss roll and projection

The classical Swiss roll is a subset of $\mathbb R^3$ parameterized by

\begin{align}t \mapsto (t \cos t,\ y,\ t \sin t),\end{align}

where $t$ ranges over an interval and $y$ is an independent coordinate.

In this script, we form a two dimensional point cloud by sampling parameters
$t$ and $y$, constructing the corresponding three dimensional points, and
then projecting to the $(x,z)$ coordinates,

\begin{align}(x,z) = (t \cos t,\ t \sin t).\end{align}

This projection produces a spiral-like planar data set. Additive Gaussian noise is
then applied to obtain a nondegenerate point cloud.

# Homology pipeline

## Delaunay complex and alpha filtration

Compute a Delaunay triangulation of the planar point cloud and take the downward closed
simplicial complex generated by its maximal simplices up to dimension 2.

For each simplex $\sigma$, define its squared alpha value

\begin{align}\alpha^2(\sigma)
   =
   \min_{c \in \mathbb R^2} \max_{x\in V(\sigma)} \|x-c\|^2.\end{align}

The alpha filtration is

\begin{align}K_u = \{\sigma : \alpha^2(\sigma) \le u\}.\end{align}

## Persistent homology

Fix the field $\mathbb F_2$. The function
:func:`homolipop.pipeline.persistent_homology_from_points` is assumed to compute a
filtration compatible simplex order, run standard boundary matrix reduction over
$\mathbb F_2$, and output a barcode.

# Visualization

The function :func:`homolipop.persistence_rings.plot_rings_from_barcode` renders the
barcode via a concentric ring representation. Styling is controlled by
:class:`homolipop.persistence_rings.RingStyle`.

# Reproducibility

All randomness is controlled by the ``seed`` parameter.


In [ ]:
from __future__ import annotations

import numpy as np

from homolipop.delaunay import delaunay_triangulation
from homolipop.pipeline import persistent_homology_from_points
from homolipop.persistence import field_Fp
from homolipop.persistence_rings import RingStyle, plot_rings_from_barcode, red_palette


def sample_swiss_roll(n: int = 600, seed: int = 0) -> np.ndarray:
    """
    Sample a noisy planar projection of a Swiss-roll point cloud.

    Parameters
    ----------
    n:
        Number of points to sample.
    seed:
        Seed for NumPy's random number generator.

    Returns
    -------
    numpy.ndarray
        Array of shape ``(n, 2)`` representing a noisy planar spiral.

    Notes
    -----
    The construction samples

    .. math::

       t = \\tfrac{3\\pi}{2} \\bigl(1 + 2U\\bigr),

    where :math:`U` is uniform on :math:`[0,1]`, and sets

    .. math::

       (x,z) = (t\\cos t,\\ t\\sin t).

    Independent Gaussian noise is added to each coordinate.
    """
    rng = np.random.default_rng(seed)
    t = 1.5 * np.pi * (1.0 + 2.0 * rng.random(n))
    x = t * np.cos(t)
    z = t * np.sin(t)
    pts = np.column_stack((x, z))
    pts = pts + rng.normal(0.0, 0.15, size=pts.shape)
    return pts


def main() -> None:
    """
    Compute alpha filtration persistent homology for a Swiss-roll projection and save a ring plot.

    The output image is written to ``swiss_roll_rings.png`` in the current working directory.
    """
    points = sample_swiss_roll()

    del_res = delaunay_triangulation(points)
    maximal_simplices = del_res.delaunay_simplices

    res = persistent_homology_from_points(
        points,
        maximal_simplices,
        max_dim=2,
        field=field_Fp(2),
    )

    style = RingStyle(cmap=red_palette(), min_width_deg=1.0, gamma=0.7)

    fig = plot_rings_from_barcode(
        res.barcode,
        ncols=3,
        style=style,
        suptitle="Swiss-roll projection, alpha filtration, F_2",
    )
    fig.savefig("swiss_roll_rings.png", dpi=220, bbox_inches="tight")


if __name__ == "__main__":
    main()